# 1. Sample Dataset

To access files stored in your Google Drive from a Google Colab notebook, you need to mount your Google Drive. The following code mounts your Google Drive to the Colab environment, allowing you to read and write files directly from your Drive.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This code extracts the contents of a ZIP file stored in your Google Drive to a specified directory in the Colab environment. The script first ensures that the destination directory exists and then unzips the file into that directory.

- `file_path`: this is the file path that contain your zipped dataset
- `destination_path`: this is the file destination to save the unzipped dataset


In [2]:
import zipfile
import os

file_path = '/content/drive/MyDrive/HistopatologyBreastCancerM400X_unhas_makassar.zip'
destination_path = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip'

# Create the destination directory if it doesn't exist
os.makedirs(destination_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_path)

print(f"Unzipped file to {destination_path}")

Unzipped file to /content/HistopatologyBreastCancerM400X_unhas_makassar.zip


## 1.1 Creating Validation Set

This code defines a function called `count_class` that counts the number of images in two different directories, representing two classes of data (e.g., benign and malignant images).

### Inputs:
- `path1`: The file path to the directory containing images for the first class (e.g., benign).
- `path2`: The file path to the directory containing images for the second class (e.g., malignant).

### Functionality:
The function calculates the number of files in each directory and prints out the count for each class as well as the total number of images in both directories combined.

The function is then called with specific paths (`path_1` and `path_2`), which point to directories within the unzipped dataset containing images of benign and malignant tumors, respectively.


In [5]:
def count_class(path1, path2):
  training_data_negative = path1
  training_data_positive = path2
  len_negative = len(os.listdir(training_data_negative))
  len_positive = len(os.listdir(training_data_positive))
  print(f'Class 1: {len_negative}')
  print(f'Class 2: {len_positive}')
  print(f'Total: {len_negative+len_positive}')

path_1 = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/train/benign'
path_2 = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/train/malignant'
count_class(path_1, path_2)

Class 1: 371
Class 2: 777
Total: 1148


This code defines a function called `create_validation_split` that creates a validation dataset by splitting off a portion of the images from the training dataset. The selected validation images are moved to a separate directory.

### Inputs:
- `train_data_dir`: The directory containing the training data, where each class has its own subdirectory.
- `validation_data_dir`: The directory where the validation data will be stored, with the same class subdirectory structure as the training data.
- `validation_split`: The proportion of the training data to move to the validation set (default is 20%).
- `seed`: A random seed to ensure reproducibility of the split (default is 123).

### Functionality:
The function:
1. Ensures the validation directory exists.
2. Iterates through each class subdirectory in the training data directory.
3. Splits the images in each class into training and validation sets based on the specified split ratio.
4. Moves the selected validation images from the training directory to the corresponding class subdirectory in the validation directory.

After running this function, the validation set will be separated from the training set, which is useful for model evaluation.


In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

def create_validation_split(train_data_dir, validation_data_dir, validation_split=0.2, seed=123):
    # Ensure the validation directory exists
    if not os.path.exists(validation_data_dir):
        os.makedirs(validation_data_dir)

    # Loop through each class directory in the training data
    for class_name in os.listdir(train_data_dir):
        class_dir = os.path.join(train_data_dir, class_name)
        if os.path.isdir(class_dir):
            # List all files in the class directory
            images = os.listdir(class_dir)
            train_images, val_images = train_test_split(images, test_size=validation_split, random_state=seed)

            # Create class directory in the validation directory
            val_class_dir = os.path.join(validation_data_dir, class_name)
            if not os.path.exists(val_class_dir):
                os.makedirs(val_class_dir)

            # Move the validation images to the validation directory
            for img_name in val_images:
                src = os.path.join(class_dir, img_name)
                dst = os.path.join(val_class_dir, img_name)
                shutil.move(src, dst)

            print(f"Moved {len(val_images)} images to {val_class_dir}")

In [7]:
training_data = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/train'
validation_data = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/validation'
create_validation_split(training_data, validation_data, validation_split=0.2, seed=123)

Moved 75 images to /content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/validation/benign
Moved 156 images to /content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/validation/malignant


In [8]:
path_1 = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/train/benign'
path_2 = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/train/malignant'
count_class(path_1, path_2)

path_1 = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/validation/benign'
path_2 = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/validation/malignant'
count_class(path_1, path_2)

path_1 = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/test/benign'
path_2 = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/test/malignant'
count_class(path_1, path_2)

Class 1: 296
Class 2: 621
Total: 917
Class 1: 75
Class 2: 156
Total: 231
Class 1: 176
Class 2: 369
Total: 545


# 2. Build Model Function

This code defines a function called `build_model_xxxx` that constructs a deep learning model, which is pre-trained on the ImageNet dataset. This function is used to build and compile a model tailored for a specific classification task.

### Inputs:
- `num_classes`: The number of output classes for the classification task. This determines the size of the final dense layer.
- `learning_rate`: The learning rate for the Adam optimizer (default is 0.001). It controls the step size during gradient descent.
- `freeze`: A boolean flag indicating whether to freeze the layers of the pre-trained Xception base model (default is `True`). When layers are frozen, their weights will not be updated during training.

### Functionality:
The function:
1. Loads the pre-trained model without the top (fully-connected) layer.
2. Adds a global average pooling layer to reduce the spatial dimensions of the output from the base model.
3. Adds a fully-connected layer with 1024 units and ReLU activation.
4. Adds a final dense layer with a softmax activation function for multi-class classification, where the number of units equals `num_classes`.
5. Optionally freezes the layers of the Xception base model to prevent their weights from being updated during training, allowing only the added layers to be trained.
6. Compiles the model using the Adam optimizer, categorical cross-entropy loss, and accuracy as a metric.

The returned model is ready to be trained on your dataset.


In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Xception
def build_model_xception(num_classes, learning_rate=0.001, freeze=True):
    base_model = Xception(weights='imagenet', include_top=False)  # Load the Xception model without the top layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Add a global spatial average pooling layer
    x = Dense(1024, activation='relu')(x)  # Add a fully-connected layer
    predictions = Dense(num_classes, activation='softmax')(x)  # Add a logistic layer for classification
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the layers of the base model
    if freeze:
      for layer in base_model.layers:
          layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# ResNet50V2
from tensorflow.keras.applications import ResNet50V2
def build_model_resnet50v2(num_classes, learning_rate=0.001, freeze=True):
    base_model = ResNet50V2(weights='imagenet', include_top=False)  # Load the ResNet50V2 model without the top layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Add a global spatial average pooling layer
    x = Dense(1024, activation='relu')(x)  # Add a fully-connected layer
    predictions = Dense(num_classes, activation='softmax')(x)  # Add a logistic layer for classification
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the layers of the base model
    if freeze:
      for layer in base_model.layers:
          layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# InceptionResNetV2
from tensorflow.keras.applications import InceptionResNetV2
def build_model_inceptionresnetv2(num_classes, learning_rate=0.001, freeze=True):
    base_model = InceptionResNetV2(weights='imagenet', include_top=False)  # Load the InceptionResNetV2 model without the top layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Add a global spatial average pooling layer
    x = Dense(1024, activation='relu')(x)  # Add a fully-connected layer
    predictions = Dense(num_classes, activation='softmax')(x)  # Add a logistic layer for classification
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the layers of the base model
    if freeze:
      for layer in base_model.layers:
          layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# DenseNet201
from tensorflow.keras.applications import DenseNet201
def build_model_densenet201(num_classes, learning_rate=0.001, freeze=True):
    base_model = DenseNet201(weights='imagenet', include_top=False)  # Load the DenseNet201 model without the top layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Add a global spatial average pooling layer
    x = Dense(1024, activation='relu')(x)  # Add a fully-connected layer
    predictions = Dense(num_classes, activation='softmax')(x)  # Add a logistic layer for classification
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the layers of the base model
    if freeze:
      for layer in base_model.layers:
          layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# EfficientNetB4
from tensorflow.keras.applications import EfficientNetB4
def build_model_efficientnetb4(num_classes, learning_rate=0.001, freeze=True):
    base_model = EfficientNetB4(weights='imagenet', include_top=False)  # Load the EfficientNetB4 model without the top layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Add a global spatial average pooling layer
    x = Dense(1024, activation='relu')(x)  # Add a fully-connected layer
    predictions = Dense(num_classes, activation='softmax')(x)  # Add a logistic layer for classification
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the layers of the base model
    if freeze:
      for layer in base_model.layers:
          layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# EfficientNetV2S
from tensorflow.keras.applications import EfficientNetV2S
def build_model_efficientnetv2s(num_classes, learning_rate=0.001, freeze=True):
    base_model = EfficientNetV2S(weights='imagenet', include_top=False)  # Load the EfficientNetV2S model without the top layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Add a global spatial average pooling layer
    x = Dense(1024, activation='relu')(x)  # Add a fully-connected layer
    predictions = Dense(num_classes, activation='softmax')(x)  # Add a logistic layer for classification
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the layers of the base model
    if freeze:
      for layer in base_model.layers:
          layer.trainable = False

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# 3. Model Training Function

This code defines a function called `train_model` that trains a deep learning model using image data from specified directories. The function handles data augmentation, model training, and the implementation of callbacks for early stopping and saving the best model.

### Inputs:
- `model`: The deep learning model to be trained, typically created using a function like `build_model_xception`.
- `train_data_dir`: The directory containing the training data, organized by class subdirectories.
- `validation_data_dir`: The directory containing the validation data, also organized by class subdirectories.
- `batch_size`: The number of samples processed before the model's weights are updated (default is 32).
- `epochs`: The number of complete passes through the training dataset (default is 10).
- `input_size`: The target size for resizing images before they are fed into the model (default is 299x299, which is suitable for the Xception model).

### Functionality:
1. **Data Augmentation**: Applies random transformations (like shearing, zooming, and flipping) to the training images to increase the diversity of the training set and reduce overfitting.
2. **Data Generators**: Creates generators for the training and validation datasets that load images in batches, apply preprocessing, and resize them to the specified `input_size`.
3. **Callbacks**:
   - **ModelCheckpoint**: Saves the model with the best validation loss during training.
   - **EarlyStopping**: Stops training if the validation loss doesn't improve for a specified number of epochs (patience is set to 5).
4. **Training**: The model is trained using the training data generator and evaluated on the validation data generator. The training history, which includes metrics like loss and accuracy over epochs, is returned.

This function is used to train the model on your dataset, with automatic handling of image data and model saving.


In [10]:
def train_model(model, train_data_dir, validation_data_dir, batch_size=32, epochs=10, input_size=(299, 299)):
    # Data augmentation for training data
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

    # Data augmentation for validation data
    validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    # Generators for training and validation data
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=input_size,
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=input_size,
        batch_size=batch_size,
        class_mode='categorical')

    # Callbacks for saving the best model and early stopping
    checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

    # Training the model
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
        epochs=epochs,
        callbacks=[checkpoint, early_stopping])

    return history

# 4. Test Model Function

This code defines a function called `test_model` that evaluates a trained deep learning model on a test dataset. It uses data augmentation specific to the Xception model's preprocessing and returns the model's performance in terms of loss and accuracy on the test data.

### Inputs:
- `model`: The trained deep learning model to be evaluated, typically loaded or passed in after training.
- `test_data_dir`: The directory containing the test data, organized by class subdirectories.
- `batch_size`: The number of samples processed at once during evaluation (default is 32).
- `input_size`: The target size for resizing test images before they are fed into the model (default is 299x299, which is suitable for the Xception model).

### Functionality:
1. **Data Augmentation**: Applies Xception's preprocessing function to the test images to ensure they are processed in the same way as the training and validation data.
2. **Test Data Generator**: Creates a generator that loads the test images in batches, applies preprocessing, resizes them to the specified `input_size`, and ensures the data is not shuffled (to maintain order for evaluation).
3. **Model Evaluation**: Evaluates the model on the test data using the generator and prints out the test loss and accuracy.

This function is used to assess how well the model generalizes to unseen data, providing metrics that summarize its performance.


In [38]:
def test_model(model, test_data_dir, batch_size=32, input_size=(299, 299)):
    # Data augmentation for test data with Xception preprocessing
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    # Generator for test data
    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=input_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

    # Load the best model
    scores = model.evaluate(test_generator)
    print(f"Test Loss: {scores[0]}")
    print(f"Test Accuracy: {scores[1]}")

# 5. Sample Run

This section of code sets up the necessary parameters for training a deep learning model using the Xception architecture and then initiates the training process.

### Parameters:
- `num_classes`: The number of output classes for the classification task, which is set to 2 for binary classification (e.g., benign vs. malignant).
- `train_data_dir`: The directory path containing the training images, organized by class subdirectories.
- `validation_data_dir`: The directory path containing the validation images, also organized by class subdirectories.
- `test_data_dir`: The directory path containing the test images, organized similarly to the training and validation directories.
- `batch_size`: The number of images processed in each training step (default is 32).
- `epochs`: The number of times the model will pass through the entire training dataset (default is 10).

### How to train a model:
1. Import the preprocessor:
  - e.g. `from tensorflow.keras.applications.xception import preprocess_input`
2. Build the model:
  - e.g. `model = build_model_xception(num_classes, learning_rate=0.001, freeze=True)`
3. Train the model:
  - e.g. `history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size=(299,299))`

The model will be saved as `best_model.keras`. Rename the model or download it before doing another training run

In [30]:
# Define parameters
num_classes = 2  # For binary classification
train_data_dir = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/train'
validation_data_dir = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/validation'
test_data_dir = '/content/HistopatologyBreastCancerM400X_unhas_makassar.zip/HistopatologyBreastCancerM400X/test'
batch_size = 32
epochs = 10


# -- Xception --
from tensorflow.keras.applications.xception import preprocess_input #Xception
input_size = (299, 299)  # Xception expects 299x299 input size
model = build_model_xception(num_classes, learning_rate=0.001, freeze=True)
history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size)

# -- ResNet50V2 --
# from tensorflow.keras.applications.resnet_v2 import preprocess_input #ResNet50V2
# input_size = (224, 224)  # ResNet50v2 expects 224x224 input size
# model = build_model_resnet50v2(num_classes, learning_rate=0.001, freeze=True)
# history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size)

# -- InceptionResNetV2 --
# from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input #InceptionResNetV2
# input_size = (299, 299)  # InceptionResNetV2 expects 299x299 input size
# model = build_model_inceptionresnetv2(num_classes, learning_rate=0.001, freeze=True)
# history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size)

# -- DenseNet201 --
# from tensorflow.keras.applications.densenet import preprocess_input #DenseNet201
# input_size = (224, 224)  # DenseNet201 expects 224x224 input size
# model = build_model_densenet201(num_classes, learning_rate=0.001, freeze=True)
# history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size)

# -- EfficientNetB4 --
# from tensorflow.keras.applications.efficientnet import preprocess_input #EfficientNetB4
# input_size = (384, 380)  # EfficientNetB4 expects 384x380 input size
# model = build_model_efficientnetb4(num_classes, learning_rate=0.001, freeze=True)
# history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size)

# -- EfficientNetV2S --
# from tensorflow.keras.applications.efficientnet_v2 import preprocess_input #EfficientNetV2S
# input_size = (384, 384)  # EfficientNetV2S expects 384x384 input size
# model = build_model_efficientnetv2s(num_classes, learning_rate=0.001, freeze=True)
# history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size)

82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Found 917 images belonging to 2 classes.
Found 231 images belonging to 2 classes.
Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 120s 2s/step - accuracy: 0.7559 - loss: 0.6230 - val_accuracy: 0.8705 - val_loss: 0.3751
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 10s 375ms/step - accuracy: 0.8750 - loss: 0.3073 - val_accuracy: 0.8571 - val_loss: 0.3230
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.8859 - loss: 0.2957 - val_accuracy: 0.8571 - val_loss: 0.3836
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.9062 - loss: 0.2432 - val_accuracy: 0.8571 - val_loss: 0.2932
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.9054 - loss: 0.2537 - val_accuracy: 0.8214 - val_loss: 0.4368
Epoch 6/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9062 - loss: 0.2605 - val_accuracy: 0.7143 - val_loss: 0.3691
Epoch 7/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.9274 - loss: 0.2132 - val_accuracy: 0.8661

### How to Evaluate a model:
1. Import the preprocessor:
  - e.g. `from tensorflow.keras.applications.xception import preprocess_input`
2. Build the model:
  - e.g. `model = build_model_xception(num_classes, learning_rate=0.001, freeze=True)`
3. Load the weights:
  - e.g. `model.load_weights('/content/best_model_xception.keras')`
4. Test the model:
  - e.g. `test_model(model, test_data_dir, batch_size=32, input_size=(299, 299))`

The performance will be printed directly. Please make sure to import the appropriate preprocessor before running `test_model`. Make sure the model built using `build_model_xxxx` matches with the model you want.

In [74]:
# -- Xception --
from tensorflow.keras.applications.xception import preprocess_input
input_size = (299, 299)
model = build_model_xception(num_classes, learning_rate=0.001, freeze=True)
model.load_weights('/content/best_model_xception.keras')
test_model(model, test_data_dir, batch_size=32, input_size=input_size)

# -- ResNet50V2 --
from tensorflow.keras.applications.resnet_v2 import preprocess_input #ResNet50V2
model = build_model_resnet50v2(num_classes, learning_rate=0.001, freeze=True)
model.load_weights('/content/best_model_resnet50v2.keras')
input_size = (224, 224)  # ResNet50v2 expects 224x224 input size
test_model(model, test_data_dir, batch_size=32, input_size=input_size)

# -- InceptionResNetV2 --
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input #InceptionResNetV2
input_size = (299, 299)  # InceptionResNetV2 expects 299x299 input size
model = build_model_inceptionresnetv2(num_classes, learning_rate=0.001, freeze=True)
model.load_weights('/content/best_model_inceptionresnetv2.keras')
test_model(model, test_data_dir, batch_size=32, input_size=input_size)

# -- DenseNet201 --
from tensorflow.keras.applications.densenet import preprocess_input #DenseNet201
input_size = (224, 224)  # DenseNet201 expects 224x224 input size
model = build_model_densenet201(num_classes, learning_rate=0.001, freeze=True)
model.load_weights('/content/best_model_densenet201.keras')
test_model(model, test_data_dir, batch_size=32, input_size=input_size)

# -- EfficientNetB4 --
from tensorflow.keras.applications.efficientnet import preprocess_input #EfficientNetB4
input_size = (384, 380)  # EfficientNetB4 expects 384x380 input size
model = build_model_efficientnetb4(num_classes, learning_rate=0.001, freeze=True)
model.load_weights('/content/best_model_efficientnetb4.keras')
test_model(model, test_data_dir, batch_size=32, input_size=input_size)

# -- EfficientNetV2S --
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input #EfficientNetV2S
input_size = (384, 384)  # EfficientNetV2S expects 384x384 input size
model = build_model_efficientnetv2s(num_classes, learning_rate=0.001, freeze=True)
model.load_weights('/content/best_model_efficientnetv2s.keras')
test_model(model, test_data_dir, batch_size=32, input_size=input_size)

Found 545 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 449ms/step - accuracy: 0.7065 - loss: 0.5790
Test Loss: 0.43257570266723633
Test Accuracy: 0.8275229334831238
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 437ms/step - accuracy: 0.7990 - loss: 0.4516
Test Loss: 0.34670519828796387
Test Accuracy: 0.853210985660553
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 24s 619ms/step - accuracy: 0.6133 - loss: 0.8042
Test Loss: 0.4448402225971222
Test Accuracy: 0.7981651425361633
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 29s 736ms/step - accuracy: 0.8460 - loss: 0.3946
Test Loss: 0.2961312532424927
Test Accuracy: 0.8880733847618103
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 26s 721ms/step - accuracy: 0.5519 - loss: 0.8069
Test Loss: 0.5145378708839417
Test Accuracy: 0.7596330046653748
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 29s 702ms/step - accuracy: 0.7696 - loss: 0.4711
Test Loss: 0.30122

### How to fine-tune a model:
1. Import the preprocessor:
  - e.g. `from tensorflow.keras.applications.xception import preprocess_input`
2. Build the model with smaller `learning_rate` and set `freeze` to false:
  - e.g. `model = build_model_xception(num_classes, learning_rate=0.0001, freeze=True)`
3. Load the weights:
  - e.g. `model.load_weights('/content/best_model_xception.keras')`
4. Train the model:
  - e.g. `history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size=(299,299))`
  
The model will be saved as `best_model.keras`. Rename the model or download it before doing another training run.

In [83]:
# # -- Xception --
# from tensorflow.keras.applications.xception import preprocess_input
# model = build_model_xception(num_classes, learning_rate=0.0001, freeze=False)
# model.load_weights('/content/best_model_xception.keras')
# input_size = (299, 299)
# history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size=input_size)

# # -- ResNet50V2 --
# from tensorflow.keras.applications.resnet_v2 import preprocess_input #ResNet50V2
# model = build_model_resnet50v2(num_classes, learning_rate=0.00001, freeze=False)
# model.load_weights('/content/best_model_resnet50v2.keras')
# input_size = (224, 224)  # ResNet50v2 expects 224x224 input size
# history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size=input_size)

# # -- InceptionResNetV2 --
# from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input #InceptionResNetV2
# input_size = (299, 299)  # InceptionResNetV2 expects 299x299 input size
# model = build_model_inceptionresnetv2(num_classes, learning_rate=0.0001, freeze=False)
# model.load_weights('/content/best_model_inceptionresnetv2.keras')
# history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size=input_size)

# # -- DenseNet201 --
# from tensorflow.keras.applications.densenet import preprocess_input #DenseNet201
# input_size = (224, 224)  # DenseNet201 expects 224x224 input size
# model = build_model_densenet201(num_classes, learning_rate=0.0001, freeze=False)
# model.load_weights('/content/best_model_densenet201.keras')
# history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size=input_size)

# # -- EfficientNetB4 --
# from tensorflow.keras.applications.efficientnet import preprocess_input #EfficientNetB4
# input_size = (384, 380)  # EfficientNetB4 expects 384x380 input size
# model = build_model_efficientnetb4(num_classes, learning_rate=0.0001, freeze=False)
# model.load_weights('/content/best_model_efficientnetb4.keras')
# history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size=input_size)

# -- EfficientNetV2S --
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input #EfficientNetV2S
input_size = (384, 384)  # EfficientNetV2S expects 384x384 input size
model = build_model_efficientnetv2s(num_classes, learning_rate=0.0001, freeze=False)
model.load_weights('/content/best_model_efficientnetv2s.keras')
history = train_model(model, train_data_dir, validation_data_dir, batch_size, epochs, input_size=input_size)

Found 917 images belonging to 2 classes.
Found 231 images belonging to 2 classes.
Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 298s 5s/step - accuracy: 0.6816 - loss: 0.5927 - val_accuracy: 0.9018 - val_loss: 0.2796
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 211ms/step - accuracy: 0.9062 - loss: 0.2387 - val_accuracy: 0.7143 - val_loss: 0.4806
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9319 - loss: 0.1750 - val_accuracy: 0.8795 - val_loss: 0.2477
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 1.0000 - loss: 0.0317 - val_accuracy: 1.0000 - val_loss: 0.0225
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9668 - loss: 0.0970 - val_accuracy: 0.9152 - val_loss: 0.2236
Epoch 6/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0623 - val_accuracy: 1.0000 - val_loss: 0.0336
Epoch 7/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9827 - loss: 0.0616 - val_accuracy: 0.9018 - val_loss: 0.2962
Epoch 8/10
28/28 ━━━━━━━━━━━━━━━

In [86]:
# -- Xception --
from tensorflow.keras.applications.xception import preprocess_input
input_size = (299, 299)
model = build_model_xception(num_classes, learning_rate=0.0001, freeze=False)
model.load_weights('/content/best_model_xception_finetuned.keras')
test_model(model, test_data_dir, batch_size=32, input_size=input_size)

# -- ResNet50V2 --
from tensorflow.keras.applications.resnet_v2 import preprocess_input #ResNet50V2
model = build_model_resnet50v2(num_classes, learning_rate=0.0001, freeze=False)
model.load_weights('/content/best_model_resnet50v2_finetuned.keras')
input_size = (224, 224)  # ResNet50v2 expects 224x224 input size
test_model(model, test_data_dir, batch_size=32, input_size=input_size)

# -- InceptionResNetV2 --
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input #InceptionResNetV2
input_size = (299, 299)  # InceptionResNetV2 expects 299x299 input size
model = build_model_inceptionresnetv2(num_classes, learning_rate=0.0001, freeze=False)
model.load_weights('/content/best_model_inceptionresnetv2_finetuned.keras')
test_model(model, test_data_dir, batch_size=32, input_size=input_size)

# -- DenseNet201 --
from tensorflow.keras.applications.densenet import preprocess_input #DenseNet201
input_size = (224, 224)  # DenseNet201 expects 224x224 input size
model = build_model_densenet201(num_classes, learning_rate=0.0001, freeze=False)
model.load_weights('/content/best_model_densenet201_finetuned.keras')
test_model(model, test_data_dir, batch_size=32, input_size=input_size)

# -- EfficientNetB4 --
from tensorflow.keras.applications.efficientnet import preprocess_input #EfficientNetB4
input_size = (384, 380)  # EfficientNetB4 expects 384x380 input size
model = build_model_efficientnetb4(num_classes, learning_rate=0.0001, freeze=False)
model.load_weights('/content/best_model_efficientnetb4_finetuned.keras')
test_model(model, test_data_dir, batch_size=32, input_size=input_size)

# -- EfficientNetV2S --
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input #EfficientNetV2S
input_size = (384, 384)  # EfficientNetV2S expects 384x384 input size
model = build_model_efficientnetv2s(num_classes, learning_rate=0.0001, freeze=False)
model.load_weights('/content/best_model_efficientnetv2s_finetuned.keras')
test_model(model, test_data_dir, batch_size=32, input_size=input_size)

Found 545 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 318 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 461ms/step - accuracy: 0.8485 - loss: 0.7053
Test Loss: 0.3737509846687317
Test Accuracy: 0.9192660450935364
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 452ms/step - accuracy: 0.8112 - loss: 0.4700
Test Loss: 0.32640472054481506
Test Accuracy: 0.8770642280578613
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 641ms/step - accuracy: 0.6538 - loss: 1.4923
Test Loss: 0.7720240950584412
Test Accuracy: 0.8220183253288269
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 29s 726ms/step - accuracy: 0.8248 - loss: 0.8370
Test Loss: 0.4183472990989685
Test Accuracy: 0.9045871496200562
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 718ms/step - accuracy: 0.8573 - loss: 0.3561
Test Loss: 0.23928412795066833
Test Accuracy: 0.9045871496200562
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 702ms/step - accuracy: 0.9382 - loss: 0.2153
Test Loss: 0.1932

# 6. Ensemble Model Function

This section of code includes functions for generating model predictions, combining probabilities (in case of an ensemble), and evaluating model accuracy. Each function is designed to handle specific tasks in the evaluation process of a deep learning model.

### Functions:

1. **`generate_output_probability`**:
   - **Purpose:** To generate predicted probabilities for the test dataset using a trained model.
   - **Inputs:**
     - `model`: The trained deep learning model used for making predictions.
     - `test_data_dir`: The directory containing the test images, organized by class subdirectories.
     - `batch_size`: The number of images processed in each prediction step (default is 32).
     - `input_size`: The target size for resizing images before they are fed into the model (default is 299x299).
   - **Output:** A numpy array of predicted probabilities for each class.

2. **`combine_output_probability`**:
   - **Purpose:** To combine probabilities from multiple models in an ensemble.
   - **Input:**
     - `probabilities`: A list or array of predicted probabilities from different models.
   - **Output:** The average of the probabilities from all models, which represents the combined prediction.

3. **`evaluate_accuracy`**:
   - **Purpose:** To calculate the accuracy of the model based on the predicted probabilities and true labels from the test dataset.
   - **Inputs:**
     - `probability`: The array of predicted probabilities for the test dataset.
     - `test_data_dir`: The directory containing the test images, organized by class subdirectories.
     - `batch_size`: The number of images processed in each evaluation step (default is 32).
     - `input_size`: The target size for resizing images before they are fed into the model (default is 299x299).
   - **Output:** The accuracy score, which indicates the proportion of correctly classified test samples.

These functions are essential for evaluating the performance of your model or ensemble, providing insights into how well your model generalizes to new, unseen data.


In [87]:
import numpy as np
from sklearn.metrics import accuracy_score

def generate_output_probability(model, test_data_dir, batch_size=32, input_size=(299, 299)):
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    # Generator for test data
    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=input_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

    # Generate probabilities
    probabilities = model.predict(test_generator)

    return probabilities

def combine_output_probability(probabilities):
    # Combine probabilities from all models
    combined_probabilities = np.mean(probabilities, axis=0)

    return combined_probabilities

def evaluate_accuracy(probability, test_data_dir, batch_size=32, input_size=(299, 299)):
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    # Generator for test data
    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=input_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

    # Get the true labels
    true_labels = test_generator.classes

    # Get the predicted labels
    predicted_labels = np.argmax(probability, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    return accuracy

# 7. Sample Run

### How to Evaluate ensemble of models:
1. Import the preprocessor:
  - e.g. `from tensorflow.keras.applications.xception import preprocess_input`
2. Build the model:
  - e.g. `model = build_model_xception(num_classes, learning_rate=0.001, freeze=True)`
3. Load the weights:
  - e.g. `model.load_weights('/content/best_model_xception_finetuned.keras')`
4. Generate probabilities:
  - e.g. `probability_xception = generate_output_probability(model, test_data_dir, batch_size=32, input_size=input_size)`
5. Combine the probabilities:
  - e.g. `combined_probabilities = combine_output_probability([probability_xception, probability_resnet50v2])`
6. Evaluate the accuracy:
  - e.g.`accuracy = evaluate_accuracy(combined_probabilities, test_data_dir, batch_size=32, input_size=input_size)`

In [88]:
from tensorflow.keras.applications.xception import preprocess_input
input_size = (299, 299)
model = build_model_xception(num_classes, learning_rate=0.0001, freeze=False)
model.load_weights('/content/best_model_xception_finetuned.keras')
probability_xception = generate_output_probability(model, test_data_dir, batch_size=32, input_size=input_size)

from tensorflow.keras.applications.resnet_v2 import preprocess_input #ResNet50V2
input_size = (224, 224)  # ResNet50v2 expects 224x224 input size
model = build_model_resnet50v2(num_classes, learning_rate=0.00001, freeze=False)
model.load_weights('/content/best_model_resnet50v2_finetuned.keras')
probability_resnet50v2 = generate_output_probability(model, test_data_dir, batch_size=32, input_size=input_size)

from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input #InceptionResNetV2
input_size = (299, 299)  # InceptionResNetV2 expects 299x299 input size
model = build_model_inceptionresnetv2(num_classes, learning_rate=0.0001, freeze=False)
model.load_weights('/content/best_model_inceptionresnetv2_finetuned.keras')
probability_inceptionresnetv2 = generate_output_probability(model, test_data_dir, batch_size=32, input_size=input_size)

from tensorflow.keras.applications.densenet import preprocess_input #DenseNet201
input_size = (224, 224)  # DenseNet201 expects 224x224 input size
model = build_model_densenet201(num_classes, learning_rate=0.0001, freeze=False)
model.load_weights('/content/best_model_densenet201_finetuned.keras')
probability_densenet201 = generate_output_probability(model, test_data_dir, batch_size=32, input_size=input_size)

from tensorflow.keras.applications.efficientnet import preprocess_input #EfficientNetB4
input_size = (384, 380)  # EfficientNetB4 expects 384x380
model = build_model_efficientnetb4(num_classes, learning_rate=0.0001, freeze=False)
model.load_weights('/content/best_model_efficientnetb4_finetuned.keras')
probability_efficientnetb4 = generate_output_probability(model, test_data_dir, batch_size=32, input_size=input_size)

from tensorflow.keras.applications.efficientnet_v2 import preprocess_input #EfficientNetV2S
input_size = (384, 384)  # EfficientNetV2S expects 384x384
model = build_model_efficientnetv2s(num_classes, learning_rate=0.0001, freeze=False)
model.load_weights('/content/best_model_efficientnetv2s_finetuned.keras')
probability_efficientnetv2s = generate_output_probability(model, test_data_dir, batch_size=32, input_size=input_size)

# combine the probabilities
combined_probabilities = combine_output_probability(
    [probability_xception,
     probability_resnet50v2,
     probability_inceptionresnetv2,
     probability_densenet201,
     probability_efficientnetb4,
     probability_efficientnetv2s]
)

# evaluate the accuracies
accuracy = evaluate_accuracy(combined_probabilities, test_data_dir, batch_size=32, input_size=input_size)
print(accuracy)


Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 481ms/step
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 503ms/step
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 899ms/step
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 924ms/step
Found 545 images belonging to 2 classes.
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 932ms/step
Found 545 images belonging to 2 classes.
0.926605504587156
